In [5]:
import pandas as pd
import hvplot.pandas
import hvplot as hv
import panel as pn

In [6]:
df = pd.read_csv(r'c:\users\zhang\world_wide_COVID_data_daily_data.csv')
df = df.sort_values(['year','month','day'])

In [7]:
continents_options = []
for continentExp in df['continentExp']:
    if continentExp not in continents_options:
        continents_options.append(continentExp)

countries_options = []
for countriesAndTerritories in df['countriesAndTerritories']:
    if countriesAndTerritories not in countries_options:
        countries_options.append(countriesAndTerritories)

In [9]:
continents = pn.widgets.MultiSelect(name='Continents', value=['Asia'],options= continents_options,)
countries = pn.widgets.MultiSelect(name='Countries', value=['China'],options= countries_options)
months = pn.widgets.IntRangeSlider(name='Months',start=1, end=12, value=(1, 2), step=1)
choices = pn.widgets.RadioButtonGroup(name= '', options=['cases', 'deaths'])

@pn.depends(continents,watch = True)
def update_countries(continents):
    df_continent = df[df['continentExp'].isin(continents)]
    countries.options = df_continent['countriesAndTerritories'].unique().tolist()
    return countries.options  

@pn.depends(continents,countries,months,choices)
def make_plot(continents,countries,months,choices):
    df_continent = df[df['continentExp'].isin(continents)]
    #countries.options = df_continent['countriesAndTerritories'].unique().tolist()
    df_country =  df_continent[df_continent['countriesAndTerritories'].isin(countries)]
    df_plot = df_country[df_country['month'].isin(range(months[0],months[1]))]
    plot = df_plot.hvplot.line(x = 'dateRep', y = choices, xlabel = 'Date',by = 'countriesAndTerritories', title = choices,rot=45,width = 600,height = 400)
    
    return plot
@pn.depends(continents,countries,months)
def make_table(continents,countries,months):
    df_continent = df[df['continentExp'].isin(continents)]
    df_country =  df_continent[df_continent['countriesAndTerritories'].isin(countries)]
    df_plot = df_country[df_country['month'].isin(range(months[0],months[1]))]
    table = df_plot.hvplot.table(['dateRep','day','month','year','continentExp','countriesAndTerritories','cases','deaths'],width = 600,height = 400)
    return table

pn.Row(pn.WidgetBox(pn.Column(continents,countries,months,choices,width=300)),make_plot,make_table).servable()

Row
    [0] WidgetBox
        [0] Column(width=300)
            [0] MultiSelect(name='Continents', options=['Asia', 'Africa', ...], value=['Asia'])
            [1] MultiSelect(name='Countries', options=['Afghanistan', ...], value=['China'])
            [2] IntRangeSlider(end=12, name='Months', start=1, value=(1, 2), value_end=2, value_start=1)
            [3] RadioButtonGroup(options=['cases', 'deaths'], value='cases')
    [1] ParamFunction(function)
    [2] ParamFunction(function)